# Linear Optimization (CS5040) Assignment 4

## Authors

| Name | Roll Number |
|-|-|
| Gautam Singh | CS21BTECH11018 |
| Varun Gupta | CS21BTECH11060 |
| Anshul Sangrame | CS21BTECH11004 |

## Setup

In [7]:
# Install libraries
%pip install numpy

# Import libraries
import numpy as np

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Parameters to run the program go here
INPUT_FILE = '../data/03/2.csv'   # Input file path
DELIMITER = ','                     # Delimiter in input file
EPS = 1e-6                          # Threshold for perturbation

## Input Handling

In [9]:
def handle_input(fname: str, delimiter: str=',') -> (np.ndarray, np.ndarray, np.ndarray, np.ndarray):
    """
    Handle input from CSV file.
    """
    # Take input from CSV file into numpy array
    input_arr = np.genfromtxt(fname, delimiter=delimiter, skip_header=0)

    # Values of A, b, c ,X
    A = input_arr[2:, :-1]
    b = input_arr[2:, -1]
    c = input_arr[1, :-1]
    X = input_arr[0,:-1]

    # Check for bad inputs, and exit if found
    if np.any(np.isnan(A)):
        raise IOError('Matrix A contains bad input:', A)
    if np.any(np.isnan(b)):
        raise IOError('Matrix b contains bad input:', b)
    if np.any(np.isnan(c)):
        raise IOError('Matrix c contains bad input:', c)
    if np.any(np.isnan(X)):
        raise IOError('Matrix z contains bad input:', X)

    # Values of m and n
    m, n = A.shape
    # Check if A is full rank
    if np.linalg.matrix_rank(A) != n:
        raise np.linalg.LinAlgError('Matrix A is not full rank:', A)

    return A,b,c,X

## Handling Degeneracy

In [10]:
# def isdegenerate(
#     A: np.ndarray,
#     b: np.ndarray,
#     c: np.ndarray,
#     X_check: np.ndarray
# ) -> bool:
#     """
#     Method to check if the given linear programming problem is degenerate.        
#     """
#     m, n = A.shape

#     # If number of tight rows is not equal to n, then the problem is degenerate.
#     if np.isclose(A@X_check, b).sum() == n:
#         return False
#     return True

# def perturb(
#     A: np.ndarray, 
#     b: np.ndarray, 
#     c: np.ndarray,
#     X_check: np.ndarray,
#     eps: float=1e-2,
#     n_iter: int=1000,
# ) -> (np.ndarray, np.ndarray, np.ndarray):
#     """
#     Method to remove degeneracy by perturbing the last m-n rows of b by a
#     small value if needed.
#     """
#     # Do not perturb if nondegenerate
#     if not isdegenerate(A, b, c,X_check):
#         return A, b, c
#     # Find dimensions of A
#     m, n = A.shape
#     # Initialize a random number generator
#     rng = np.random.default_rng()
#     # Perturb for certain number of iterations
#     while n_iter:
#         _b = b
#         # Choose a small value to add at random
#         eps_rng = rng.uniform(eps, 10*eps)
#         b[n:] += eps_rng**np.arange(1,m-n+1)
#         # Return if not degenerate
#         if not isdegenerate(A, b, c,X_check):
#             return A, _b, c
#         n_iter -= 1
#     # Raise error if perturbation fails
#     raise RuntimeError('Could not perturb the linear programming problem')

def perturb (
    b: np.ndarray,
    eps: np.ndarray=1e-4,
) -> np.ndarray:
    """
    Perturb for certain number of iterations. Choose small values to add at
    random.
    """
    # Initialize random number generator
    rng = np.random.default_rng()
    # Add random values
    eps_rng = rng.uniform(eps, 2*eps, b.shape)
    b1 = b + eps_rng
    return b1

## Finding the Optimal Vertex

In [11]:
def vertex_directions(
    A: np.ndarray,
    b: np.ndarray,
    v: np.ndarray,
) -> np.ndarray or None:
    """
    Function to find directions of the other vertices of the polytope from given
    vertex.
    """
    tight_rows = np.where(np.isclose(A@v, b))
    A1 = A[tight_rows]
    if A1.shape[0] == A1.shape[1]:
        return -np.linalg.inv(A1.T)
    else:
        return None

def simplex_neighbour(
    A: np.ndarray,
    b: np.ndarray,
    c: np.ndarray,
    u: np.ndarray,
) -> np.ndarray or bool:
    """
    Function to find a neighbouring vertex with greater cost, or report that
    there is no such neighbour by returning True. In case of an error, False is
    returned. 
    """
    # Find directions to other vertices
    z = vertex_directions(A, b, u)

    # Check if perturbation is needed
    if z is None:
        return False

    # Find costs for each direction
    costs = z@c

    # Find directions which give positive cost
    costs_positive = np.where(costs > 0)[0]
    
    # If there are no such directions, declare optimality
    if len(costs_positive) == 0:
        return True
    else:
        # Get any direction with positive cost
        v = z[costs_positive[0]]

        # Check for unboundedness. If A@v keeps decreasing in that direction,
        # then the LP is unbounded.
        if len(np.where(A@v > 0)[0]) == 0:
            raise np.linalg.LinAlgError('LP is unbounded.')

        # Find untight rows
        untight_rows = np.where(~np.isclose(A@u, b))
        A2 = A[untight_rows]
        b2 = b[untight_rows]

        # Find feasible neighbour and required coefficients
        # Coefficients are (b2 - A2@u)/(A2@v)
        alpha = (b2-A2@u)/(A2@v)
        t = np.min(alpha[alpha >= 0])
        return u + t*v

def simplex(
    A: np.ndarray,
    b: np.ndarray,
    c: np.ndarray,
    u: np.ndarray,
    n_iter: int=10000,
) -> (bool, list):
    """ 
    Function to implement the simplex algorithm.
    """
    steps = []
    while n_iter:
        # Display vertex and cost
        steps.append([u, c.T@u])
        # Find neighbour of a greater cost
        u1 = simplex_neighbour(A, b, c, u)
        # Return in case of error or optimality
        if isinstance(u1, bool):
            return u1, steps
        else:
            u = u1
        n_iter -= 1
    return False

## Driver 

In [12]:
if __name__ == "__main__":
    # Number of times the simplex algorithm must be run with perturbation.
    n_iter = 1000
    try:
        A, _b, c , u = handle_input(INPUT_FILE, DELIMITER)
        # Perturb on b, original in _b
        b = np.empty_like(_b)
        b[:] = _b
        for i in range(n_iter):
            res, steps = simplex(A, b, c, u)
            if res:
                # Simplex algorithm has converged
                print(f"[INFO] Simplex algorithm converged in iteration {i+1}.")
                # Print the steps
                for j in range(len(steps)):
                    print("Iteration", j+1, ": x =", steps[j][0], "cost =", steps[j][1])
                break
            else:
                # Perturb by small amounts and try again
                b = perturb(_b)
                # Alert the user about perturbation
                print(f"[INFO] Degeneracy detected in iteration {i+1}. LP has been perturbed.")
    except BaseException as error:
        print(f'An exception occurred: {error}')

[INFO] Degeneracy detected in iteration 1. LP has been perturbed.
[INFO] Degeneracy detected in iteration 2. LP has been perturbed.
[INFO] Degeneracy detected in iteration 3. LP has been perturbed.
[INFO] Degeneracy detected in iteration 4. LP has been perturbed.
[INFO] Degeneracy detected in iteration 5. LP has been perturbed.
[INFO] Degeneracy detected in iteration 6. LP has been perturbed.
[INFO] Degeneracy detected in iteration 7. LP has been perturbed.
[INFO] Degeneracy detected in iteration 8. LP has been perturbed.
[INFO] Degeneracy detected in iteration 9. LP has been perturbed.
[INFO] Degeneracy detected in iteration 10. LP has been perturbed.
[INFO] Degeneracy detected in iteration 11. LP has been perturbed.
[INFO] Degeneracy detected in iteration 12. LP has been perturbed.
[INFO] Degeneracy detected in iteration 13. LP has been perturbed.
[INFO] Degeneracy detected in iteration 14. LP has been perturbed.
[INFO] Degeneracy detected in iteration 15. LP has been perturbed.
[INF

[INFO] Degeneracy detected in iteration 606. LP has been perturbed.
[INFO] Degeneracy detected in iteration 607. LP has been perturbed.
[INFO] Degeneracy detected in iteration 608. LP has been perturbed.
[INFO] Degeneracy detected in iteration 609. LP has been perturbed.
[INFO] Degeneracy detected in iteration 610. LP has been perturbed.
[INFO] Degeneracy detected in iteration 611. LP has been perturbed.
[INFO] Degeneracy detected in iteration 612. LP has been perturbed.
[INFO] Degeneracy detected in iteration 613. LP has been perturbed.
[INFO] Degeneracy detected in iteration 614. LP has been perturbed.
[INFO] Degeneracy detected in iteration 615. LP has been perturbed.
[INFO] Degeneracy detected in iteration 616. LP has been perturbed.
[INFO] Degeneracy detected in iteration 617. LP has been perturbed.
[INFO] Degeneracy detected in iteration 618. LP has been perturbed.
[INFO] Degeneracy detected in iteration 619. LP has been perturbed.
[INFO] Degeneracy detected in iteration 620. LP 